In [1]:
!pip install pandas

In [2]:
import pandas as pd
import numpy as np
import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!pip install sklearn
from sklearn import linear_model

Solving environment: ...working... done

# All requested packages already installed.



In [21]:
#Pull UK house purchase price data. Limit to recent purchases in London.
min_date = "1+February+2019"
county = "LONDON"
csv_link = "http://landregistry.data.gov.uk/app/ppd/ppd_data.csv?county={}&et%5B%5D=lrcommon%3Afreehold&et%5B%5D=lrcommon%3Aleasehold&header=true&limit=all&min_date={}&nb%5B%5D=true&nb%5B%5D=false&ptype%5B%5D=lrcommon%3Adetached&ptype%5B%5D=lrcommon%3Asemi-detached&ptype%5B%5D=lrcommon%3Aterraced&ptype%5B%5D=lrcommon%3Aflat-maisonette&ptype%5B%5D=lrcommon%3AotherPropertyType&tc%5B%5D=ppd%3AstandardPricePaidTransaction&tc%5B%5D=ppd%3AadditionalPricePaidTransaction".format(county, min_date)

df = pd.read_csv(csv_link, header=None)

#Update column headers
headers = ['unique_id','price_paid','deed_date','postcode','property_type','new_build','estate_type','saon','paon','street','locality','town','district','county','transaction_category','linked_data_uri']
df.columns = headers

print(df.shape)

(2308, 16)


In [23]:
#Limit to flats purchased in London, that are not new builds

df = df[df['transaction_category']=='A']
df = df[df['property_type']=='F']
df = df[df['new_build']=='N']
#df = df[df['county']=='GREATER LONDON']

#Drop unwanted columns
df.drop(columns=['unique_id','deed_date', 'property_type', 'new_build','estate_type','saon','paon','locality','district','transaction_category','linked_data_uri'], inplace=True)

#Drop duplicate postcodes
df.drop_duplicates(subset=['postcode'], inplace=True)

#df = df[0:5]

print(df.shape)
df.head()

(777, 5)


,price_paid,postcode,street,town,county
5,240000,BR1 3AG,FREELANDS ROAD,BROMLEY,GREATER LONDON
6,315000,BR1 3AX,WIDMORE ROAD,BROMLEY,GREATER LONDON
13,280000,BR1 3TY,OAKLANDS ROAD,BROMLEY,GREATER LONDON
24,200000,BR2 0AB,BROMLEY ROAD,BROMLEY,GREATER LONDON
25,280000,BR2 0JW,BECKENHAM GROVE,BROMLEY,GREATER LONDON


In [24]:
#Add lontitude and lattitude data
addresses = df['postcode']
latitude = []
longitude = []

for address in addresses:
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address+", UK", timeout=2)
    try:
        latitude.append(location.latitude)
        longitude.append(location.longitude)
    except:
        latitude.append(0)
        longitude.append(0)
        
df['Latitude']= latitude
df['Longitude']= longitude

In [25]:
#remove regions that location information was not available
df = df[(df['Latitude']!=0)| (df['Longitude']!=0)]

#df.head(10)
print(df.shape[0])

769


In [26]:
#Foursquare credentials
CLIENT_ID = 'QKP4KVQQZPVXMOKNIXHGQAT1E2IJZCHQPFB4XJGP3MC0CFUO' # your Foursquare ID
CLIENT_SECRET = '1N5BIRBR1EA5CO1QFMNHU1241SOFM5KJLLQRJGMDC0NOYWPH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 200 # limit of number of venues returned by Foursquare API
radius = 100 # define radius
categoryId = ''

In [27]:
#What are the most popular places nearby?

def getNearbyVenues(names, latitudes, longitudes, avg_price, radius=100):
   
    region_list = []
    price_list = []
    
    for name, lat, lng, price in zip(names, latitudes, longitudes, avg_price):
                    
        # create the API request URL
        #url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(    
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            categoryId)
        
        #print(url) 
        
        # make the GET request
        results = requests.get(url).json()["response"]
        
        #flatten JSON file
        resultsVen = json_normalize(results['venues'])
        resultsCat = json_normalize(data=results['venues'], record_path=['categories'], record_prefix='cat_', errors='ignore')
        resultsVenCat = pd.concat([resultsVen, resultsCat], axis=1)
        resultsVenCat = resultsVenCat[['name','location.distance','location.formattedAddress','location.lat','location.lng','cat_name']]
               
        #Add region name and average price 
        for i in range(0,resultsVen.shape[0]):
            region_list.append(name)
            price_list.append(price)
        
        region_list = pd.DataFrame(region_list, columns=['Property Postcode'])
        price_list = pd.DataFrame(price_list, columns=['Property Price'])
        
        nearbyVenues = pd.concat([region_list, price_list, resultsVenCat], axis=1)

    return(nearbyVenues)

In [28]:
london_venues = pd.DataFrame()

#request venue information for each postcode
for region in df['postcode']:
    region_venues = df[df['postcode']==region]
    london_venues = london_venues.append((getNearbyVenues(names=region_venues['postcode'],
                                                  latitudes=region_venues['Latitude'],
                                                  longitudes=region_venues['Longitude'],
                                                  avg_price=region_venues['price_paid'],
                                                  radius=radius)),ignore_index=True)
    
#rename columns
london_venues.rename(columns={'name':'Venue Name',
                          'location.distance':'Venue Distance',
                          'location.formattedAddress':'Venue Address',
                          'location.lat':'Venue Lat',
                          'location.lng':'Venue Lng',
                          'cat_name':'Venue Category'}, inplace=True)

print(london_venues.shape)
london_venues.head()

,Property Postcode,Property Price,Venue Name,Venue Distance,Venue Address,Venue Lat,Venue Lng,Venue Category
0,BR1 3AG,240000,Village Stores,239,"[Bromley, Greater London, BR1 3HZ, United King...",51.410290,0.024560,Convenience Store
1,BR1 3AG,240000,wagamama,627,"[intu Shopping Centre (High St), Bromley, Grea...",51.404025,0.017960,Asian Restaurant
2,BR1 3AG,240000,Alan de Maid,622,"[The Old Municipal Buildings, 19 East Street, ...",51.406587,0.015505,Real Estate Office
3,BR1 3AG,240000,Sharps Bedrooms,499,"[24/30 Widmore Road, Bromley, Greater London, ...",51.406037,0.017762,Furniture / Home Store
4,BR1 3AG,240000,Coast,642,"[Unit 245 The Glades (High St), Bromley, Great...",51.404722,0.016671,Boutique


In [30]:
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe

london_onehot['Property Price'] = london_venues['Property Price'] 
london_onehot['Property Postcode'] = london_venues['Property Postcode'] 

#Reorder columns
fixed_columns = [london_onehot.columns[-1]] + [london_onehot.columns[-2]] + list(london_onehot.columns[:-2])
london_onehot = london_onehot[fixed_columns]

df1 = london_onehot.groupby(['Property Postcode','Property Price']).sum()
df1.reset_index(inplace=True)

print(df1.shape)
df1.head()

(769, 649)


,Property Postcode,Property Price,ATM,Accessories Store,Acupuncturist,Adult Boutique,Adult Education Center,Advertising Agency,Afghan Restaurant,African Restaurant,...,Whisky Bar,Windmill,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo Exhibit
0,BR1 3AG,240000,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,4,0,0,0
1,BR1 3AX,315000,0,1,0,0,2,0,0,1,...,0,0,0,0,0,0,4,0,0,0
2,BR1 3TY,280000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,BR2 0AB,200000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,BR2 0JW,280000,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [70]:
pd.set_option('display.max_columns', 100)
df1[df1['College Cricket Pitch']>0]

,Property Postcode,Property Price,ATM,Accessories Store,Acupuncturist,Adult Boutique,Adult Education Center,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Alternative Healer,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Art Studio,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Astrologer,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Baggage Locker,Bakery,Ballroom,Bank,Bar,Baseball Field,Basketball Court,Bath House,...,Town Hall,Toy / Game Store,Track,Track Stadium,Trade School,Trail,Trailer Park,Train Station,Tram Station,Transportation Service,Travel & Transport,Travel Agency,Travel Lounge,Tree,Tunnel,Turkish Restaurant,University,Urgent Care Center,Used Bookstore,Vacation Rental,Vape Store,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Village,Vineyard,Volleyball Court,Voting Booth,Warehouse,Warehouse Store,Waste Facility,Watch Shop,Water Park,Waterfront,Wedding Hall,Weight Loss Center,Well,Whisky Bar,Windmill,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo Exhibit
524,SW13 9EA,400000,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,3,0,0,0,0,0,0,1,1,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
657,TW12 1ND,624950,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0


In [43]:
#Create dataframe of feature list
df_features = pd.DataFrame(df1.columns)
df_features = df_features.iloc[2:df1.shape[1], :]
df_features.reset_index(inplace=True)

In [65]:
regr = linear_model.LinearRegression()

#Set venues as features
x = df1.iloc[:, 2:df1.shape[1]]

#Set price as result(y)
y=df1[['Property Price']]

#Fit data
regr.fit(x, y)

#transpose feature coefficients and store in a dataframe
coefs = pd.DataFrame(regr.coef_).T

#Add feature labels and sort
df_coef = pd.concat([df_features, coefs],axis=1,ignore_index=True)
df_coef.drop(columns=0, inplace=True)
df_coef.rename(columns={1:'Venue Category',2:'Coefficient'}, inplace=True)
df_coef.sort_values(by=['Coefficient'],inplace=True)

print ('Coefficients: ', regr.coef_)
print ('Intercept: ',regr.intercept_)

Coefficients:  [[-3.88853703e+05 -4.32084081e+04 -1.56628207e+05 -3.03025303e+05
   1.15032012e+05  7.92065021e+04  1.21491790e+05  2.39810259e+04
  -2.73131110e+05  1.19556631e+06 -3.21202937e+05 -1.62447396e+05
   4.24408728e+05  1.94498334e+05 -9.46968172e+04 -1.13500999e+05
  -1.16592701e+04 -1.78957221e+04  1.94910673e+05 -3.74175888e+04
   3.56535006e+05 -3.68729026e+04 -2.40953293e+05 -1.52474277e+05
   3.06214400e+04 -2.18923669e+05  5.85797763e+03  6.22680736e+04
  -1.07662745e+05 -1.32699855e+04  3.25205858e+04  3.22346415e+05
   4.00077409e+05  6.40983504e+04 -2.51116068e+04  8.69532229e+04
  -1.41137678e+04 -6.29076071e+04  1.16460709e+04 -3.47661124e+04
  -1.46512813e+05  2.56135115e+04  9.36515901e+03 -1.59508952e+04
   1.22934985e+04 -2.26776540e+05 -4.46851231e+04  1.95882937e+06
   3.38427812e+05  2.83972879e+05  1.00965811e+05  1.36355286e+04
   5.26097035e+04 -9.91026398e+03  2.32511686e+05 -3.75359685e+05
  -5.07603613e+04  7.92340637e+04 -4.34508222e+04  3.94850320

In [67]:
pd.set_option('display.max_rows', 100)
#df_coef['Coefficient'] = df_coef['Coefficient'].round(0)
df_coef

,Venue Category,Coefficient
342,Laboratory,-3.567550e+06
483,Rafting,-3.012561e+06
447,Pet Café,-1.297878e+06
460,Planetarium,-1.205528e+06
152,College Stadium,-1.134334e+06
120,Circus,-1.079164e+06
291,Heliport,-1.005282e+06
389,Monastery,-6.726973e+05
84,Burrito Place,-5.631901e+05
632,Water Park,-5.385881e+05


In [87]:
%% bash
git clone https://github.com/OpenDataServices/flatten-tool.git
cd flatten-tool
virtualenv -p $(which python3) .ve
source .ve/bin/activate
pip install -r requirements_dev.txt

UsageError: Cell magic `%%` not found.


In [83]:
!pip install -r requirements_dev.txt

url1 = 'https://api.foursquare.com/v2/venues/categories?&client_id=QKP4KVQQZPVXMOKNIXHGQAT1E2IJZCHQPFB4XJGP3MC0CFUO&client_secret=1N5BIRBR1EA5CO1QFMNHU1241SOFM5KJLLQRJGMDC0NOYWPH&v=20180605'

results1 = requests.get(url1).json()["response"]
#print(results1)

    
#flatten JSON file
resultsCat1 = json_normalize(results1['categories'])
#print(resultsCat1)

#resultsCat2 = json_normalize(data=results1['categories'], record_path=['categories','categories'], record_prefix='cat_', errors='ignore')
#print(resultsCat2)
#resultsVenCat = pd.concat([resultsVen, resultsCat], axis=1)
#resultsVenCat = resultsVenCat[['name','location.distance','location.formattedAddress','location.lat','location.lng','cat_name']]    

SyntaxError: invalid syntax (<ipython-input-83-c1c3475c3264>, line 1)